In [1]:
import pandas as pd

In [27]:
#read in file for dataframe
#df = pd.read_stata('CapstoneCoding\ER_Visits.dta') 
df = pd.read_stata('CapstoneCoding/ER_Visits.dta')
df.head(3)

,DUID,PID,DUPERSID,EVNTIDX,EVENTRN,ERHEVIDX,FFEEIDX,PANEL,MPCDATA,ERDATEYR,...,ERDOF23X,ERDSL23X,ERDWC23X,ERDOT23X,ERDXP23X,ERDTC23X,IMPFLAG,PERWT23F,VARSTR,VARPSU
0,2790032,106,2790032106,2790032106001001,3 ROUND 3,-1,-1,27 PANEL 27,1 HAS MPC DATA,2023,...,0.0,0.0,0.0,0.0,192.52,1822.0,2 COMPLETE MPC DATA,12162.846840,2064,3
1,2790037,101,2790037101,2790037101007201,3 ROUND 3,-1,-1,27 PANEL 27,2 NO MPC DATA,2023,...,0.0,0.0,0.0,0.0,386.75,1361.0,3 FULLY IMPUTED,9676.522242,2104,3
2,2790043,101,2790043101,2790043101003001,4 ROUND 4,-1,-1,27 PANEL 27,1 HAS MPC DATA,2023,...,0.0,0.0,0.0,0.0,288.53,1357.0,2 COMPLETE MPC DATA,29407.390290,2045,3


In [ ]:
#investigate dataset
df.info()

In [28]:
#create df with select columns
ER_df = df[['DUPERSID','ERDATEMM','ERDMD23X','ERDMR23X','ERDPV23X', 'EVNTIDX', 'EVENTRN',
            'ERFMD23X','ERFMR23X','ERFPV23X', 'VSTRELCN']].copy()
ER_df.head()

,DUPERSID,ERDATEMM,ERDMD23X,ERDMR23X,ERDPV23X,EVNTIDX,EVENTRN,ERFMD23X,ERFMR23X,ERFPV23X,VSTRELCN
0,2790032106,1,91.87,0.00,0.00,2790032106001001,3 ROUND 3,256.59,0.00,0.00,1 YES
1,2790037101,1,0.00,385.31,1.44,2790037101007201,3 ROUND 3,0.00,196.49,50.12,1 YES
2,2790043101,6,0.00,286.50,2.03,2790043101003001,4 ROUND 4,0.00,574.11,146.46,1 YES
3,2790043101,8,0.00,8.60,0.00,2790043101003501,5 ROUND 5,0.00,500.14,127.59,1 YES
4,2790043103,1,0.00,0.00,0.00,2790043103002201,3 ROUND 3,0.00,0.00,0.00,1 YES


In [29]:
#change column names from codebook variables
ER_df.rename(columns={'DUPERSID': 'person_id', 'EVNTIDX' : 'visit_id', 'EVENTRN' : 'round', 'VSTRELCN': 'visit condition',
                      'ERDATEMM': 'month','ERDMD23X':'dr_medicaid',
                      'ERDMR23X': 'dr_medicare',
                      'ERDPV23X': 'dr_private','ERFMD23X':'f_medicaid','ERFMR23X': 'f_medicare',
                      'ERFPV23X' : 'f_private'}, inplace=True)
ER_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4241 entries, 0 to 4240
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   person_id        4241 non-null   object  
 1   month            4241 non-null   category
 2   dr_medicaid      4241 non-null   float64 
 3   dr_medicare      4241 non-null   float64 
 4   dr_private       4241 non-null   float64 
 5   visit_id         4241 non-null   object  
 6   round            4241 non-null   category
 7   f_medicaid       4241 non-null   float64 
 8   f_medicare       4241 non-null   float64 
 9   f_private        4241 non-null   float64 
 10  visit condition  4241 non-null   category
dtypes: category(3), float64(6), object(2)
memory usage: 278.4+ KB


In [30]:
#aggregate column of payment
#df['sum_cols'] = df['col1'] + df['col2']
ER_df['medicaid'] = ER_df['dr_medicaid'] + ER_df['f_medicaid']
ER_df['medicare'] = ER_df['dr_medicare'] + ER_df['f_medicaid']
ER_df['private'] = ER_df['dr_private'] + ER_df['f_private']

In [31]:
#drop columns from df
ER_df = ER_df.drop(['dr_medicaid', 'dr_medicare', 'dr_private',
                   'f_medicare', 'f_medicaid', 'f_private'], axis=1)
ER_df.head()

,person_id,month,visit_id,round,visit condition,medicaid,medicare,private
0,2790032106,1,2790032106001001,3 ROUND 3,1 YES,348.46,256.59,0.00
1,2790037101,1,2790037101007201,3 ROUND 3,1 YES,0.00,385.31,51.56
2,2790043101,6,2790043101003001,4 ROUND 4,1 YES,0.00,286.50,148.49
3,2790043101,8,2790043101003501,5 ROUND 5,1 YES,0.00,8.60,127.59
4,2790043103,1,2790043103002201,3 ROUND 3,1 YES,0.00,0.00,0.00


In [32]:
#change month dtype
ER_df['month'] = ER_df['month'].astype(int)
ER_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4241 entries, 0 to 4240
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   person_id        4241 non-null   object  
 1   month            4241 non-null   int64   
 2   visit_id         4241 non-null   object  
 3   round            4241 non-null   category
 4   visit condition  4241 non-null   category
 5   medicaid         4241 non-null   float64 
 6   medicare         4241 non-null   float64 
 7   private          4241 non-null   float64 
dtypes: category(2), float64(3), int64(1), object(2)
memory usage: 207.6+ KB


In [ ]:
ER_df.head(2)

In [33]:
event_df= pd.read_stata('condition_event.dta')
event_df.head(3)

,DUPERSID,CONDIDX,EVNTIDX,CLNKIDX,EVENTYPE,PANEL
0,2790002101,2790002101001,2790002101001201,27900021010012790002101001201,1 OFFICE-BASED MEDICAL PROVIDER VISIT,27 PANEL 27
1,2790002101,2790002101001,2790002101001801,27900021010012790002101001801,1 OFFICE-BASED MEDICAL PROVIDER VISIT,27 PANEL 27
2,2790002101,2790002101001,2790002101002303,27900021010012790002101002303,8 PRESCRIBED MEDICINE,27 PANEL 27


In [34]:
#select only columns needed for merge to find condition
event_df = event_df[['EVNTIDX', 'CONDIDX']].copy()

#change names of columns
event_df.rename(columns={'EVNTIDX': 'visit_id', 'CONDIDX': 'condition_id'}, inplace=True)

In [ ]:
event_df.head(3)

In [35]:
#read in medical condition
condition_df = pd.read_stata('CapstoneCoding/Medical_Conditionsta/MedicalConditons.dta') 

condition_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63656 entries, 0 to 63655
Data columns (total 29 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   DUID      63656 non-null  object  
 1   PID       63656 non-null  int16   
 2   DUPERSID  63656 non-null  object  
 3   CONDN     63656 non-null  int8    
 4   CONDIDX   63656 non-null  object  
 5   PANEL     63656 non-null  category
 6   CONDRN    63656 non-null  category
 7   AGEDIAG   63656 non-null  category
 8   CRND1     63656 non-null  category
 9   CRND2     63656 non-null  category
 10  CRND3     63656 non-null  category
 11  CRND4     63656 non-null  category
 12  CRND5     63656 non-null  category
 13  INJURY    63656 non-null  category
 14  ACCDNWRK  63656 non-null  category
 15  ICD10CDX  63656 non-null  object  
 16  CCSR1X    63656 non-null  object  
 17  CCSR2X    63656 non-null  object  
 18  CCSR3X    63656 non-null  object  
 19  CCSR4X    63656 non-null  object  
 20  HHCOND

In [36]:
#select column for condition to connect condition and er_df
condition_df = condition_df[['CONDIDX','ICD10CDX']].copy()

In [37]:
#rename columns
condition_df.rename(columns={'CONDIDX':'condition_id', 'ICD10CDX':'ICD10'}, inplace=True)
condition_df.head()

,condition_id,ICD10
0,2790002101001,E11
1,2790002101004,R03
2,2790002101005,L30
3,2790002101006,E11
4,2790002101008,E55


In [ ]:
#merge er_df with event to eventually get to condition
ER_df = ER_df.merge(event_df, on='visit_id', how='left')

#merge with condition
ER_df = ER_df.merge(condition_df, on='condition_id', how='left')

#drop duplicates at the visit and condition level together
ER_df= ER_df.drop_duplicates(subset=['visit_id', 'condition_id'])

#the ICD10 with -15 values are unknown conditions and will be deleted
ER_df = ER_df[ER_df['ICD10'] != '-15']



,person_id,month,visit_id,round,visit condition,medicaid,medicare,private,condition_id,ICD10
1,2790032106,1,2790032106001001,3 ROUND 3,1 YES,348.46,256.59,0.00,2790032106004,A08
2,2790037101,1,2790037101007201,3 ROUND 3,1 YES,0.00,385.31,51.56,2790037101019,U07
3,2790043101,6,2790043101003001,4 ROUND 4,1 YES,0.00,286.50,148.49,2790043101017,J42


In [ ]:
#sample new ER df
ER_df.head(5)

In [43]:
#count conditions per visit
condition_count = ER_df.groupby('visit_id')['condition_id'].nunique().reset_index(name='num_conditions')

In [ ]:
#single visits by condition
single_condition = condition_count[condition_count['num_conditions'] ==1]['visit_id']
ER_df = ER_df[ER_df['visit_id'].isin(single_condition)].copy()


In [47]:
ER_df.head(2)

,person_id,month,visit_id,round,visit condition,medicaid,medicare,private,condition_id,ICD10
1,2790032106,1,2790032106001001,3 ROUND 3,1 YES,348.46,256.59,0.00,2790032106004,A08
2,2790037101,1,2790037101007201,3 ROUND 3,1 YES,0.00,385.31,51.56,2790037101019,U07


In [48]:
ER_df = ER_df[['ICD10','medicaid', 'medicare','private']]
ER_df.head(3)

,ICD10,medicaid,medicare,private
1,A08,348.46,256.59,0.00
2,U07,0.00,385.31,51.56
3,J42,0.00,286.50,148.49
